## **Pavyzdžiai: Multi-AI agentai viešbučio rezervavimui**

Šiandienos sparčiai besikeičiančiame pasaulyje verslo kelionės planavimas apima ne tik skrydžio ir viešbučio kambario rezervavimą. Tam reikia tam tikro koordinavimo ir efektyvumo lygio, kurį kartais sunku pasiekti. Čia įsijungia Multi-AI agentai, kurie keičia mūsų kelionių valdymo būdus.

Įsivaizduokite turintys komandą intelektualių agentų, dirbančių kartu, kad tiksliai ir lengvai pasirūpintų kiekvienu jūsų kelionės aspektu. Naudodamiesi mūsų pažangia AI technologija, sukūrėme specializuotus agentus rezervavimo paslaugoms ir maršruto sudarymui, užtikrindami sklandžią ir be streso kelionių patirtį.

Tai yra bazinis scenarijus. Planuojant verslo kelionę, reikia pasitarti su verslo kelionių agentu dėl skrydžio bilietų informacijos, viešbučio informacijos ir pan. Per AI agentus galime sukurti agentus rezervavimo paslaugoms ir agentus maršruto sudarymui, kurie bendradarbiautų ir pagerintų intelekto lygį.


# Inicializuokite Azure AI Agent Service ir gaukite konfigūracijos informaciją iš **.env**

### **.env**

Sukurkite .env failą

**.env** faile yra Azure AI Agent Service prisijungimo eilutė, AOAI naudojamas modelis ir atitinkamas Google API Search paslaugos API, ENDPOINT ir kt.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Jūsų Azure AI Agent Service modelio diegimo pavadinimas"

[**PASTABA**] Reikia modelio su 100 000 Rate Limit (ženklų per minutę) ir 600 Rate Limit (užklausų per minutę)

  Modelį galite gauti Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Jūsų Azure AI Agent Service projekto prisijungimo eilutė"

  Projekto prisijungimo eilutę galite rasti savo projekto apžvalgoje AI Foundry Portal ekrane.

- **SERPAPI_SEARCH_API_KEY** = "Jūsų SERPAPI Search API RAKTAS"
- **SERPAPI_SEARCH_ENDPOINT** = "Jūsų SERPAPI Search Endpoint"

Norėdami gauti Azure AI Agent Service Model Deployment Name ir Project Connection String, turite sukurti Azure AI Agent Service. Rekomenduojama naudoti [šį šabloną](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json), kad sukurtumėte ją tiesiogiai （***Pastaba:*** Azure AI Agent Service šiuo metu nustatytas ribotame regione. Rekomenduojama atsižvelgti į [šį saitą](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support), kad nustatytumėte regioną）

Agentui reikia prieigos prie SERPAPI. Rekomenduojama registruotis naudojant [šį saitą](https://serpapi.com/searches). Po registracijos galite gauti unikalų API RAKTĄ ir ENDPOINTą


# Prisijungimas prie Azure

Dabar turite prisijungti prie Azure. Atidarykite terminalą VScode ir vykdykite komandą `az login`


# Setup 

Norėdami paleisti šį užrašų knygelę, turėsite įdiegti šias bibliotekas. Čia pateikiamas reikalingų bibliotekų sąrašas ir atitinkamos pip diegimo komandos:

azure-identity: Azure autentifikacijai.
requests: HTTP užklausoms siųsti.
semantic-kernel: Semantic kernel sistemai (jei tai yra specifinė ar pritaikyta biblioteka, gali reikėti ją įdiegti iš konkretaus šaltinio ar saugyklos).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Paaiškinimas: 
import asyncio: Tai importuoja asyncio modulį, kuris suteikia asinchroninio programavimo palaikymą Python kalboje. Jis leidžia rašyti lygiagrečią kodą naudojant async ir await sintaksę.
from typing import Annotated: Tai importuoja Annotated tipą iš typing modulio. Annotated naudojamas pridėti metaduomenis tipo užuominoms, kas gali būti naudinga įvairiems tikslams, pavyzdžiui, validavimui, dokumentacijai ar įrankiams


In [ ]:
import asyncio,os
from typing import Annotated

# Paaiškinimas:
Naudodami from dotenv import load_dotenv ir load_dotenv(), galite lengvai tvarkyti konfigūracijos nustatymus ir jautrią informaciją (pvz., API raktus ir duomenų bazės URL) .env faile, laikydami juos atskirai nuo jūsų šaltinio kodo ir taip padarydami savo programą saugesnę bei lengviau konfigūruojamą.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Paaiškinimas:

Importo pareiškimas: from azure.identity.aio import DefaultAzureCredential: Tai importuoja DefaultAzureCredential klasę iš azure.identity.aio modulio. aio modulio pavadinime nurodo, kad jis skirtas asinchroninėms operacijoms.

DefaultAzureCredential paskirtis: DefaultAzureCredential klasė yra Azure SDK dalis Python kalbai. Ji suteikia numatytą būdą autentifikuotis Azure paslaugose. Ji bando autentifikuotis naudojant kelis metodus tam tikra tvarka, pavyzdžiui, naudojant aplinkos kintamuosius, valdomą tapatybę ir Azure CLI kredencialus.

Asinchroninės operacijos: aio modulis rodo, kad DefaultAzureCredential klasė palaiko asinchronines operacijas. Tai reiškia, kad ją galite naudoti kartu su asyncio neblokuojantiems autentifikavimo užklausoms atlikti.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Paaiškinimas:
Importuoja įvairius modulius ir klases iš semantic_kernel paketo. Štai kiekvieno importo paaiškinimas:

AgentGroupChat iš semantic_kernel.agents: Ši klasė valdo funkcionalumus, susijusius su grupiniu pokalbiu AI agentams. AzureAIAgent ir AzureAIAgentSettings iš semantic_kernel.agents.azure_ai

AzureAIAgent: Ši klasė naudojama kurti ir valdyti AI agentus, kurie naudoja Azure AI paslaugas.

AzureAIAgentSettings: Ši klasė naudojama konfigūruoti nustatymus AzureAIAgent agentui. TerminationStrategy iš semantic_kernel.agents.strategies.termination.termination_strategy:

Ši klasė apibrėžia strategijas, kaip nutraukti AI agentų vykdymą tam tikromis sąlygomis. ChatMessageContent iš semantic_kernel.contents.chat_message_content:

Ši klasė naudojama tvarkyti pokalbių žinučių turinį.
AuthorRole iš semantic_kernel.contents.utils.author_role:

Ši klasė apibrėžia skirtingas autorių roles pokalbių žinučių kontekste.

kernel_function iš semantic_kernel.functions.kernel_function_decorator: Šis dekoratorius naudojamas apibrėžti kernelio funkcijas, kurios yra funkcijos, galinčios būti vykdomos semantic_kernel sistemoje.
Šie importai paruošia reikalingus komponentus kurti ir valdyti AI agentus, galinčius bendrauti grupiniame pokalbyje, pavyzdžiui, užsakant viešbučius ar atliekant panašias užduotis.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Paaiškinimas:
Toliau mes importuojame CodeInterpreterTool klasę iš azure.ai.projects.models modulio.

CodeInterpreterTool: Ši klasė yra Azure AI SDK dalis ir naudojama kodui interpretuoti ir vykdyti AI projektų kontekste. Ji teikia funkcionalumus kodo fragmentams paleisti, kodo analizei arba kodo vykdymo integravimui AI darbo eigos procesuose.
Šis importas paruošia būtiną komponentą naudoti CodeInterpreterTool jūsų projekte, kuris gali būti naudingas užduotims, susijusioms su kodo dinamišku interpretavimu ir vykdymu.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Paaiškinimas: 
ApprovalTerminationStrategy klasė suteikia specifinę strategiją AI agento veiklos nutraukimui. Agentas bus nutrauktas, jei paskutinėje jo sąveikos istorijoje esančioje žinutėje bus žodis „saved“. Tai gali būti naudinga situacijose, kai agento užduotis laikoma baigta, kai gaunamas patvirtinimas, kad kažkas buvo „saved“. Apibrėžkite sąveikos metodą. Po rezervacijos plano išsaugojimo, jis gali būti sustabdytas gavus signalą „saved“.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Paaiškinimas:

Kodo eilutė inicijuoja AzureAIAgentSettings objektą su numatytaisiais arba iš anksto nustatytais parametrais, iškviesdama create() metodą. Šis nustatymų objektas (ai_agent_settings) gali būti naudojamas AzureAIAgent egzemplioriaus konfigūravimui ir valdymui.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Paaiškinimas:
Importuodami requests biblioteką, galite lengvai siųsti HTTP užklausas ir sąveikauti su interneto paslaugomis savo Python kode.


In [ ]:
import requests

# Paaiškinimas:
Tai yra kintamasis, kuriame saugomas API raktas, skirtas prieiti prie SERP (Paieškos variklio rezultatų puslapio) API paslaugos. API raktas yra unikalus identifikatorius, naudojamas autentikuoti užklausas, susijusias su jūsų paskyra.

'GOOGLE_SEARCH_API_KEY': Tai yra laikinis simbolių eilutės žymeklis. Turite pakeisti ''GOOGLE_SEARCH_API_KEY' savo tikru SERP API raktu.

Paskirtis: Šios eilutės paskirtis – saugoti API raktą kintamajame, kad jis galėtų būti naudojamas autentifikuoti užklausas į SERP API paslaugą. API raktas reikalingas norint prieiti prie paslaugos ir atlikti paiešką.

Kaip gauti SERP API raktą: Norėdami gauti SERP API raktą, atlikite šiuos bendrus veiksmus adresu https://serpapi.com (tikslūs veiksmai gali skirtis priklausomai nuo konkrečios naudojamos SERP API paslaugos):

Pasirinkite SERP API paslaugą: Yra keletas SERP API paslaugų, pavyzdžiui, SerpAPI, Google Custom Search JSON API ir kitos. Pasirinkite tą, kuri geriausiai atitinka jūsų poreikius.

Užsiregistruokite paskyrai:

Eikite į pasirinktos SERP API paslaugos svetainę https://www.serpapi.com ir užsiregistruokite paskyrai. Gali prireikti pateikti tam tikrą pagrindinę informaciją ir patvirtinti savo el. pašto adresą.

Sukurkite API raktą:

Užsiregistravę prisijunkite prie savo paskyros ir eikite į API skyrių arba informacijos suvestinę. Ieškokite opcijos sukurti arba sugeneruoti naują API raktą.
Kopijuokite API raktą:

Kai API raktas bus sugeneruotas, jį nukopijuokite. Šis raktas bus naudojamas autentifikuoti jūsų užklausas į SERP API paslaugą.
Pakeiskite žymeklį:

Pakeiskite žymeklį savo .env faile


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Paaiškinimas:
BookingPlugin klasė suteikia metodus viešbučiams ir skrydžiams užsakyti naudojantis Serpapi.com Google paieškos API. Ji sukuria reikalingus parametrus, siunčia API užklausas ir apdoroja atsakymus, kad pateiktų atitinkamą užsakymo informaciją. API raktas (SERPAPI_SEARCH_API_KEY) ir galinis taškas (SERPAPI_SEARCH_ENDPOINT) naudojami autentifikavimui ir užklausų siuntimui Google paieškos API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Paaiškinimas:
SavePlugin klasė suteikia metodą saving_plan, skirtą kelionės planams išsaugoti naudojant Azure AI paslaugas. Ji nustato Azure autentifikacijos duomenis, sukuria AI agentą, apdoroja vartotojo įvestis norint sugeneruoti ir išsaugoti kelionės plano turinį bei tvarko failo išsaugojimą ir valymo operacijas. Metodas grąžina "Saved" po sėkmingo užbaigimo.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Paaiškinimas:
Šis kodas nustato Azure AI agentus, kurie tvarko skrydžių ir viešbučių rezervaciją bei kelionių planų išsaugojimą pagal vartotojo įvestis. Jis naudoja Azure prisijungimo duomenis agentams sukurti ir sukonfigūruoti, apdoroja vartotojo įvestis per grupinį pokalbį ir užtikrina tinkamą išvalymą po užduočių atlikimo. Agentai naudoja specifinius papildinius (BookingPlugin ir SavePlugin) tam, kad atliktų savo užduotis.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors stengiamės užtikrinti tikslumą, prašome atkreipti dėmesį, kad automatizuoti vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojame naudoti profesionalų žmogaus atliekamą vertimą. Mes neatsakome už jokius nesusipratimus ar neteisingus aiškinimus, kylančius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
